In [ ]:
! pip install -q evaluate torch tqdm datasets peft transformers rouge_score
! pip install -U bitsandbytes

  Preparing metadata (setup.py) ... done


In [ ]:
import os
import yaml
import json
import torch
import evaluate
from tqdm import tqdm
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline


In [ ]:
DATASETS_DIR = "./datasets"

In [ ]:
def select_subset(dataset, n_samples, seed=42):
    """
    Select a subset of the dataset.
    If n_samples is "all" or None, return the entire dataset.
    Otherwise, sample n_samples examples.
    """
    if n_samples == "all" or n_samples is None:
        return dataset

    if n_samples > len(dataset):
        print(f"⚠️  Requested {n_samples} samples but only {len(dataset)} available. Using all samples.")
        return dataset

    return dataset.shuffle(seed=seed).select(range(n_samples))


def load_and_prepare_dataset(cfg):
    """
    Load dataset splits according to configuration.
    Ensures the FULL dataset is cached, and subsets are selected per run.
    Supports both new-style ("dataset": {"splits": {...}}) and old-style (top-level keys) configs.
    """
    # -----------------------------------------------------------------------
    # Extract dataset configuration
    # -----------------------------------------------------------------------
    if "dataset" in cfg:
        cfg_dataset = cfg["dataset"]
        dataset_name = cfg_dataset["name"]
        splits_cfg = cfg_dataset.get("splits", {})
        n_train = splits_cfg.get("train", "all")
        n_val = splits_cfg.get("validation", "all")
        n_test = splits_cfg.get("test", "all")
        seed = cfg_dataset.get("seed", 42)
    elif "datasets" in cfg and isinstance(cfg["datasets"], list):
        cfg_dataset = cfg["datasets"][0]
        dataset_name = cfg_dataset["path"]
        n_train = cfg.get("train_samples", "all")
        n_val = cfg.get("val_samples", "all")
        n_test = cfg.get("test_samples", "all")
        seed = cfg.get("seed", 42)
    else:
        raise KeyError("Dataset configuration not found. Expected 'dataset' or 'datasets' key.")

    # -----------------------------------------------------------------------
    # Load or download full dataset
    # -----------------------------------------------------------------------
    os.makedirs(DATASETS_DIR, exist_ok=True)
    local_path = os.path.join(DATASETS_DIR, dataset_name.replace("/", "_"))

    if os.path.exists(local_path):
        print(f"📂 Loading dataset from local cache: {local_path}")
        dataset = load_from_disk(local_path)
    else:
        print(f"⬇️  Downloading dataset from Hugging Face: {dataset_name}")
        dataset = load_dataset(dataset_name)
        dataset.save_to_disk(local_path)
        print(f"✅ Full dataset saved locally to: {local_path}")

    # -----------------------------------------------------------------------
    # Handle variations in split keys and select subsets dynamically
    # -----------------------------------------------------------------------
    val_key = "validation" if "validation" in dataset else "val"

    train = select_subset(dataset["train"], n_train, seed=seed)
    val = select_subset(dataset[val_key], n_val, seed=seed)
    test = select_subset(dataset["test"], n_test, seed=seed)

    print(f"📊 Loaded {len(train)} train / {len(val)} val / {len(test)} test samples (from full cache).")
    return train, val, test


In [ ]:
def setup_model_and_tokenizer(cfg, use_4bit: bool = None, use_lora: bool = None):
    """
    Load model, tokenizer, and apply quantization + LoRA config if specified.

    Args:
        cfg (dict): Configuration dictionary containing:
            - base_model
            - quantization parameters
            - lora parameters (optional)
            - bf16 or fp16 precision
        use_4bit (bool, optional): Override whether to load in 4-bit mode.
        use_lora (bool, optional): Override whether to apply LoRA adapters.

    Returns:
        tuple: (model, tokenizer)
    """
    model_name = cfg["base_model"]
    print(f"\nLoading model: {model_name}")

    # ------------------------------
    # Tokenizer setup
    # ------------------------------
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    # Determine quantization + LoRA usage
    load_in_4bit = use_4bit if use_4bit is not None else cfg.get("load_in_4bit", False)
    apply_lora = use_lora if use_lora is not None else ("lora_r" in cfg)

    # ------------------------------
    # Quantization setup (optional)
    # ------------------------------
    quant_cfg = None
    if load_in_4bit:
        print("⚙️  Enabling 4-bit quantization...")
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type=cfg.get("bnb_4bit_quant_type", "nf4"),
            bnb_4bit_use_double_quant=cfg.get("bnb_4bit_use_double_quant", True),
            bnb_4bit_compute_dtype=getattr(
                torch, cfg.get("bnb_4bit_compute_dtype", "bfloat16")
            ),
        )
    else:
        print("⚙️  Loading model in full precision (no quantization).")

    # ------------------------------
    # Model loading
    # ------------------------------
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quant_cfg,
        device_map="auto",
        dtype=(
            torch.bfloat16
            if cfg.get("bf16", True) and torch.cuda.is_available()
            else torch.float32
        ),
    )

    # ------------------------------
    # LoRA setup (optional)
    # ------------------------------
    if apply_lora:
        print("🔧 Applying LoRA configuration...")
        model = prepare_model_for_kbit_training(model)
        lora_cfg = LoraConfig(
            r=cfg.get("lora_r", 8),
            lora_alpha=cfg.get("lora_alpha", 16),
            target_modules=cfg.get("target_modules", ["q_proj", "v_proj"]),
            lora_dropout=cfg.get("lora_dropout", 0.05),
            bias="none",
            task_type="CAUSAL_LM",
        )
        model = get_peft_model(model, lora_cfg)
        model.print_trainable_parameters()
    else:
        print("🔹 Skipping LoRA setup — using base model only.")

    return model, tokenizer

In [ ]:
CONFIG_FILE_PATH = "./config.yaml"

def load_config(config_path: str = CONFIG_FILE_PATH):
    """
    Load and parse a YAML configuration file.

    Args:
        config_path (str): Path to the config file.

    Returns:
        dict: Parsed configuration dictionary.
    """
    with open(config_path, "r", encoding="utf-8") as f:
        cfg = yaml.safe_load(f)
    return cfg


In [ ]:
def generate_predictions(
    model,
    tokenizer,
    dataset,
    task_instruction,
    num_samples=None,
    batch_size=8,
    max_new_tokens=256,
):
    """
    Generate model predictions for a dataset (e.g., summaries).

    Args:
        model: The loaded model (base or fine-tuned).
        tokenizer: Corresponding tokenizer.
        dataset: Hugging Face dataset split containing 'dialogue' and 'summary'.
        task_instruction (str): Instruction prefix for generation.
        num_samples (int, optional): Number of samples to evaluate.
        batch_size (int): Number of examples per inference batch.
        max_new_tokens (int): Max tokens to generate per sample.

    Returns:
        list[str]: Generated summaries.
    """
    if num_samples is not None and num_samples < len(dataset):
        dataset = dataset.select(range(num_samples))

    # Prepare prompts
    prompts = []
    for sample in dataset:
        user_prompt = (
            f"{task_instruction}\n\n"
            f"## Dialogue:\n{sample['dialogue']}\n"
            "## Summary:"
        )
        messages = [{"role": "user", "content": user_prompt}]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    # Initialize pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        dtype="auto",
        do_sample=False,
    )

    # Generate predictions
    preds = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating summaries"):
        batch = prompts[i : i + batch_size]
        outputs = pipe(batch, max_new_tokens=max_new_tokens, return_full_text=False)
        preds.extend([o[0]["generated_text"].strip() for o in outputs])

    return preds


def compute_rouge(predictions, samples):
    """
    Compute ROUGE scores between predictions and reference summaries.

    Args:
        predictions (list[str]): Model-generated outputs.
        samples (datasets.Dataset): Dataset containing reference 'summary' field.

    Returns:
        dict: ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    rouge = evaluate.load("rouge")
    references = [s["summary"] for s in samples]
    return rouge.compute(predictions=predictions, references=references)


In [ ]:
"""
evaluate_baseline.py
Evaluate the base (unfine-tuned) model on the SAMSum dataset to establish baseline ROUGE scores.
"""

cfg = load_config()

def evaluate_baseline():
    """Run baseline evaluation on the SAMSum dataset using the base model."""

    # Load validation data
    _, val_data, _ = load_and_prepare_dataset(cfg)
    print(f"📊 Loaded {len(val_data)} validation samples.")

    # Load model + tokenizer (no quantization or LoRA)
    model, tokenizer = setup_model_and_tokenizer(
        cfg=cfg,
        use_4bit=False,
        use_lora=False,
    )

    # Generate predictions
    preds = generate_predictions(
        model=model,
        tokenizer=tokenizer,
        dataset=val_data,
        task_instruction=cfg["task_instruction"],
        batch_size=4,
    )

    # Compute ROUGE metrics
    scores = compute_rouge(preds, val_data)

    # -----------------------------------------------------------------------
    # Save outputs
    # -----------------------------------------------------------------------
    results = {
        "model_name": cfg["base_model"],
        "num_samples": len(val_data),
        "rouge1": scores["rouge1"],
        "rouge2": scores["rouge2"],
        "rougeL": scores["rougeL"],
    }

    results_path = "eval_results.json"
    preds_path = "predictions.jsonl"

    with open(results_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    with open(preds_path, "w", encoding="utf-8") as f:
        for i, pred in enumerate(preds):
            json.dump(
                {
                    "dialogue": val_data[i]["dialogue"],
                    "reference": val_data[i]["summary"],
                    "prediction": pred,
                },
                f,
            )
            f.write("\n")

    print(f"\n💾 Saved results to: {results_path}")
    print(f"💾 Saved predictions to: {preds_path}")

    return scores, preds


# ---------------------------------------------------------------------------
# Main
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    print("🚀 Starting baseline evaluation...")
    rouge_scores, predictions = evaluate_baseline()
    print("\n✅ Evaluation complete.")


    print("\n📈 Baseline ROUGE Results:")
    print(f"  ROUGE-1: {rouge_scores['rouge1']:.2%}")
    print(f"  ROUGE-2: {rouge_scores['rouge2']:.2%}")
    print(f"  ROUGE-L: {rouge_scores['rougeL']:.2%}")

    print("\nExample prediction:\n")
    print(predictions[0])
    print("\nRouge scores:\n")
    print(rouge_scores)


🚀 Starting baseline evaluation...
📂 Loading dataset from local cache: ./datasets/knkarthick_samsum
📊 Loaded 14731 train / 200 val / 200 test samples (from full cache).
📊 Loaded 200 validation samples.

Loading model: meta-llama/Llama-3.2-1B-Instruct
⚙️  Loading model in full precision (no quantization).


Device set to use cuda:0


🔹 Skipping LoRA setup — using base model only.


Generating summaries: 100%|██████████| 50/50 [02:38<00:00,  3.18s/it]



💾 Saved results to: eval_results.json
💾 Saved predictions to: predictions.jsonl

✅ Evaluation complete.

📈 Baseline ROUGE Results:
  ROUGE-1: 35.46%
  ROUGE-2: 11.18%
  ROUGE-L: 26.56%

Example prediction:

Victoria and Magda commiserated about their financial struggles, with Victoria expressing frustration about overspending, and Magda jokingly commiserating about her car insurance being paid for the rest of the year.

Rouge scores:

{'rouge1': np.float64(0.3545633958887797), 'rouge2': np.float64(0.1117964129289076), 'rougeL': np.float64(0.26562850525807696), 'rougeLsum': np.float64(0.2660814302845086)}
